In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import run_eval,train,prepare
from models.KNRM import KNRMModel

hparams = {
    'mode':'demo',
    'name':'knrm',
    'train_embedding': True,
    'save_each_epoch':False,
    'epochs':7,
    'batch_size':50,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'embedding_dim':300,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'device':'cuda:0',
    'attrs': ['title'],
}

In [2]:
save_path = 'models/model_params/{}_{}_{}'.format(hparams['name'],hparams['mode'],hparams['epochs']) +'.model'
device = torch.device(hparams['device']) if torch.cuda.is_available() else torch.device("cpu")

vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)
knrmModel = KNRMModel(vocab=vocab,hparams=hparams).to(device)

In [3]:
train(knrmModel, hparams, loader_train, loader_test, save_path, loader_validate)

training...
epoch 0 , step 30 , loss: 1.6539: : 40it [00:05,  6.98it/s]
epoch 1 , step 30 , loss: 1.6383: : 40it [00:05,  7.58it/s]
epoch 2 , step 30 , loss: 1.6375: : 40it [00:05,  7.69it/s]
epoch 3 , step 30 , loss: 1.6461: : 40it [00:05,  7.73it/s]
epoch 4 , step 30 , loss: 1.6426: : 40it [00:05,  7.62it/s]
epoch 5 , step 30 , loss: 1.6429: : 40it [00:05,  7.78it/s]
epoch 6 , step 30 , loss: 1.6389: : 40it [00:05,  7.03it/s]
0it [00:00, ?it/s]save success!
testing...
362it [00:09, 39.57it/s]
0it [00:00, ?it/s]evaluation results:{'group_auc': 0.5249, 'ndcg@5': 0.254, 'ndcg@10': 0.3151, 'mean_mrr': 0.2373}
validating...
1506it [00:35, 42.38it/s]
evaluation results:{'group_auc': 0.511, 'ndcg@5': 0.2368, 'ndcg@10': 0.2972, 'mean_mrr': 0.2306}
